<a href="https://colab.research.google.com/github/timbosssds/ML/blob/main/Hyde2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# This are extracts from Hyde (Kaggle)

In [13]:
# -- RAG set up

# RAG Setup

In [14]:
# --- Install lib
#!pip3 install --upgrade google-cloud-aiplatform
#!pip3 install ipython pandas[output_formatting] google-cloud-language==2.10.0
!pip install langchain
!pip install langchain-community

!pip install PyPDF2
!pip install pypdf
#!pip install sentence_transformers
!pip install sentence-transformers
!pip install torch
!pip install pandas
!pip install pdfplumber
!pip install llama_index

In [15]:
# -- Version 0a -- (trialing llama_index SentenceSplitter)
# Can i split by page + chunk

import os as os
import pandas as pd
pd.set_option('display.max_colwidth', 750)
from llama_index.core.node_parser import SentenceSplitter
import pdfplumber
from langchain.document_loaders import PDFPlumberLoader

def load_pdfs_from_directory(directory_path, max_chunk_size=1000):
    docs = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            loader = PDFPlumberLoader(file_path)
            loaded_docs = loader.load()

            # Use LlamaIndex SentenceSplitter
            text_splitter = SentenceSplitter()
            for doc in loaded_docs:
                sentences = text_splitter.split_text(doc.page_content)
                for sentence in sentences:
                    # Sub-chunk by character length (max 1000)
                    chunks = [sentence[i:i+max_chunk_size] for i in range(0, len(sentence), max_chunk_size)]
                    for chunk in chunks:
                        docs.append({
                            'page_content': chunk,  # Changed key to 'chunk'
                            'metadata': doc.metadata,
                            'len': len(chunk)
                    })
    return docs

#directory_path = "/home/jupyter/pdf"
#directory_path = "/kaggle/input/benpayfactsandcs/"
#directory_path = "/kaggle/input/hoemloanbooklet/"
directory_path = "/home"



all_pdf_docs = load_pdfs_from_directory(directory_path)

# Create DataFrame directly with desired columns
df1 = pd.DataFrame(all_pdf_docs)

# No further processing needed, df already has required columns

display(df1.head(2))

,page_content,metadata,len
0,Home Loan Booklet.\n9 November 2023\n1,"{'source': '/home/home-loan-booklet.pdf', 'file_path': '/home/home-loan-booklet.pdf', 'page': 0, 'total_pages': 23, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231030150924+10'00'', 'ModDate': 'D:20231030150945+10'30'', 'Author': 'Sharon Polkinghorne', 'Title': 'Home Loan Booklet', 'Company': 'Bendigo and Adelaide Bank', 'PADocID': '34514846v1'}",36
1,Bendigo and Adelaide Bank Limited Table of contents\nThe Bendigo Centre\nHome Loan terms and conditions 1\nBendigo VIC 3550\nTelephone 1300 236 344 1. What we lend and when 2\nABN 11 068 049 178. 2. Interest charges 2\nAustralian Credit Licence 237879\n3. Joint Accounts 3\n4. Fees and charges 3\nBanking products are products of Bendigo and\n5. What you owe us 4\nAdelaide Bank.\n6. Repayments 4\n7. Redraw facility 4\nHome Loan Booklet\n8. Sweep facility 5\nAbout this contract 9. Building Loans 5\n10. If you are in default 6\nThis document does not contain all of the information\nwe must give you before you enter into this contract. 11. What happens to payments we\n6\nreceive\nThe rest of the information is in the Loan Schedule.\nThe cont...,"{'source': '/home/home-loan-booklet.pdf', 'file_path': '/home/home-loan-booklet.pdf', 'page': 1, 'total_pages': 23, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231030150924+10'00'', 'ModDate': 'D:20231030150945+10'30'', 'Author': 'Sharon Polkinghorne', 'Title': 'Home Loan Booklet', 'Company': 'Bendigo and Adelaide Bank', 'PADocID': '34514846v1'}",1000


In [16]:
# # # -- Investigation (for Version 0a)
# #df1 = df.copy()
# print('shape', df1.shape)
# pd.set_option('display.max_colwidth', None)
# dfa = df1[['chunk','metadata', 'len']]
# dfa['chunk_org'] = dfa['chunk'].str.lower()
# df_filtered = dfa[dfa['chunk_org'].str.contains('easy money')]
# display(df_filtered.head())

In [17]:
# pd.set_option('display.max_colwidth', None)
# df_filtered1 = df_filtered[['chunk']]
# display(df_filtered1.head())

# Model set up

In [18]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate

In [19]:
#!pip install --upgrade sentence-transformers
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the Sentence Transformer model
model_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

print(df1.columns)
try:
  # Attempt to access the column
    embeddings = model_trans.encode(df1['page_content'])
except KeyError:
    # Handle missing column or data (e.g., print error message)
    print("Error: 'page_content' column not found in df1.")
    # You can choose to exit or impute missing values here

#page_content
# Embed the chunked text
embeddings = model_trans.encode(df1['page_content'])
#embeddings = model_trans.encode(df1['filtered_text'])

# Convert embedding vectors into one-dimensional arrays
one_dimensional_embeddings = []
for embedding in embeddings:
    one_dimensional_embedding = np.ravel(embedding)
    one_dimensional_embeddings.append(one_dimensional_embedding)

# Combine the original text and embeddings into a DataFrame
data = {
    "original_text": df1['page_content'],
    #"original_text": df1['filtered_text'],
    "embeddings": one_dimensional_embeddings
}

dfe = pd.DataFrame(data)
df3 = pd.concat([df1, dfe], axis=1)
df3.drop(['original_text'], axis=1, inplace=True)
display(df3.head(3))


Index(['page_content', 'metadata', 'len'], dtype='object')


,page_content,metadata,len,embeddings
0,Home Loan Booklet.\n9 November 2023\n1,"{'source': '/home/home-loan-booklet.pdf', 'file_path': '/home/home-loan-booklet.pdf', 'page': 0, 'total_pages': 23, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231030150924+10'00'', 'ModDate': 'D:20231030150945+10'30'', 'Author': 'Sharon Polkinghorne', 'Title': 'Home Loan Booklet', 'Company': 'Bendigo and Adelaide Bank', 'PADocID': '34514846v1'}",36,"[-0.0121377, -0.029595556, -0.024513813, -0.06316285, -0.018697811, 0.015847584, -0.00029816953, 0.049123965, 0.008452071, 0.067275316, 0.0077778385, 0.01895175, 0.07799541, 0.007562824, 0.034652192, -0.021292768, 0.030150902, 0.023461368, -0.04202788, -0.041856837, -0.0038764186, 0.035890535, -0.046219483, 0.045761958, -0.056909803, -0.02113933, -0.0048837312, -0.027655711, -0.0068402644, -0.011573563, 0.03352008, 0.04338819, 0.038213395, -0.035741817, 1.5851025e-06, -0.02778065, -0.016926432, -0.00051096105, -0.054177288, 0.056806892, -0.0206828, 0.0076194913, 0.022016458, 0.0010489542, -0.040820092, -0.022308424, 0.053387847, 0.06970775, 0.015992325, 0.027213626, 0.009325661, -0.05358543, 0.024056364, -0.044356845, 0.054719895, -0.02..."
1,Bendigo and Adelaide Bank Limited Table of contents\nThe Bendigo Centre\nHome Loan terms and conditions 1\nBendigo VIC 3550\nTelephone 1300 236 344 1. What we lend and when 2\nABN 11 068 049 178. 2. Interest charges 2\nAustralian Credit Licence 237879\n3. Joint Accounts 3\n4. Fees and charges 3\nBanking products are products of Bendigo and\n5. What you owe us 4\nAdelaide Bank.\n6. Repayments 4\n7. Redraw facility 4\nHome Loan Booklet\n8. Sweep facility 5\nAbout this contract 9. Building Loans 5\n10. If you are in default 6\nThis document does not contain all of the information\nwe must give you before you enter into this contract. 11. What happens to payments we\n6\nreceive\nThe rest of the information is in the Loan Schedule.\nThe cont...,"{'source': '/home/home-loan-booklet.pdf', 'file_path': '/home/home-loan-booklet.pdf', 'page': 1, 'total_pages': 23, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231030150924+10'00'', 'ModDate': 'D:20231030150945+10'30'', 'Author': 'Sharon Polkinghorne', 'Title': 'Home Loan Booklet', 'Company': 'Bendigo and Adelaide Bank', 'PADocID': '34514846v1'}",1000,"[-0.03783013, -0.03396031, 0.009471221, -0.024542907, -0.0080953315, 0.042903375, 0.0366529, -0.01408062, 0.054187164, 0.045738727, 0.008299247, -0.08039027, 0.039762378, 0.020773835, -0.012273284, 0.018059282, -0.01407603, -0.01795256, 0.010880633, -0.025975587, 0.0017982612, 0.040984794, -0.0065805684, 0.03338702, -0.034220595, 0.025709193, 0.02454841, -0.01302081, 0.013301669, 0.0010308812, 0.013085375, 0.01478043, -0.019062005, -0.035431106, 2.772996e-06, -0.038424283, 0.010668585, -0.011223461, -0.098905586, 0.075449996, -0.03711771, -0.016922848, 0.028802453, 0.0140390955, -0.06237985, -0.01549355, 0.03839068, -0.006268712, 0.0022983293, 0.02819235, -0.012863055, -0.040891755, -0.04916461, -0.03843471, 0.085238375, -0.06999578, -0..."
2,"ombination 7\n17. How we may exercise our rights 7\n18. Our certificates 8\n19. Assignment 8\n20. Valuations 8\n21. Blanks 8\n22. Notices, other communications and\n8\nserving documents\n23. Changes 8\n24. Waiver 10\n25. Set-off 10\n26. Consumer Credit legislation and\n10\nseverance\n27. Applicable law 10\n28. Anti-Money Laundering & Counter-\n10\nTerrorism Financing (AML/CTF)\n29. Security - not applicable 10\n30. Electronic Banking 10\n31. Meaning of words 16\nThings you should know about your\n19\nproposed credit contract\n1","{'source': '/home/home-loan-booklet.pdf', 'file_path': '/home/home-loan-booklet.pdf', 'page': 1, 'total_pages': 23, 'Creator': 'Power PDF Create', 'Producer': 'Power PDF Create', 'CreationDate': 'D:20231030150924+10'00'', 'ModDate': 'D:20231030150945+10'30'', 'Author': 'Sharon Polkinghorne', 'Title': 'Home Loan Booklet', 'Company': 'Ben

In [2]:
#!pip install transformers
#!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [1]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Check for Hugging Face token in environment variable
if "HF_TOKEN" not in os.environ:
  print("Error: Please set the environment variable HF_TOKEN with your Hugging Face token.")
  exit()

try:
  tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-7b-it")
  model = AutoModelForCausalLM.from_pretrained(
      "google/gemma-1.1-7b-it",
      device_map="auto",
      torch_dtype=torch.bfloat16
  )

  #input_text = "Write me a poem about Machine Learning."
  input_text = "How to win a taekwondo tournament?."
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  outputs = model.generate(**input_ids)
  print(tokenizer.decode(outputs[0]))

except HTTPError as e:
  print(f"An error occurred while downloading the model: {e}")
  print("Please check your internet connection or model name.")



Error: Please set the environment variable HF_TOKEN with your Hugging Face token.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>How to win a taekwondo tournament?.

**1. Physical Preparation:**

* Intense training


In [ ]:
# -- Above 3 cells combined to function (but keeping above 3 if I need to work on the separately)

# Above cell, converted to function
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 600)
from sentence_transformers import SentenceTransformer

def answer_question(user, df3):
    question_embedding = model_trans.encode(user, convert_to_tensor=True).cpu()
    similarities = df3['embeddings'].apply(lambda x: np.dot(question_embedding, x))

    most_similar_indices = similarities.nlargest(10).index
    #most_similar_indices = similarities.nlargest(4).index

    most_similar_rows = df3.iloc[most_similar_indices]

    #cont = most_similar_rows['page_content_data'].tolist()
    cont = most_similar_rows['page_content'].tolist()
    #cont = most_similar_rows['page_content_org'].tolist()
    context = ' '.join(cont)

#     # -- Inference (Vertex version)
#     system = f"""You are an AI Agent specialized to answer to questions about the context provided.\
#     In order to create the answer, please only use the information from the\
#     context provided (Context). Do not include other information.\
#     Answer with simple words.\
#     If needed, include also explanations.\
#     Context: {context}"""

    #     # -- Inference (Kaggle 1)
    system = f"""You are an AI Agent specialized to answer to questions about the context provided.\
    In order to create the answer, please only use the information from the\
    context provided (Context). Do not include other information.\
    Context: {context}"""

        # -- Inference (KAggle 2)
#     #system = f"""You are an AI Agent specialized to answer to questions about the context provided.\
#     system = f"""You are an AI Agent specialized in finding the passages of text in {context} that most accurately answer questions you receive.\
#                  Your task is then to answer the question using only passages you located and not any other information."""

    #user = "Write a Python code to display text in a star pattern."
    #user = question
    #user = "can Foreign Currency Transactions performed?"

    prompt = f"System: {system} \n User: {user} \n AI: "

    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")


    # -- Experiment -------------------------------------------------------------------------
    # top_p: try slightly lower values (e.g., 0.9) to nudge the model towards more informative retrievals.
    # top_k: try adjusting it within a reasonable range (e.g., 8-16) to find a balance.
    # ---------------------------------------------------------------------------------------

    #outputs = model.generate(**inputs, max_length=1500, temperature = 0.25, num_return_sequences=1)
    #outputs = model.generate(**inputs, max_length=1500, temperature = 0.25, top_p = 0.025, top_k = 40, num_return_sequences=1, do_sample=True)
    #outputs = model.generate(**inputs, max_length=1500, temperature = 0.15, top_p = 1.0, top_k = 10, num_return_sequences=1, do_sample=True)

    #outputs = model.generate(**inputs, max_new_tokens=1500, temperature = 0.10, top_p = 1.0, top_k = 12, num_return_sequences=1, do_sample=True)
    outputs = model.generate(**inputs, max_new_tokens=1500, temperature = 0.10, top_p = 0.8, top_k = 8, num_return_sequences=1, do_sample=True)

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    #Markdown(text.split("AI:")[1])
    out1 = (text.split("AI:")[1])
    return out1
    return most_similar_rows

#user = "Can i you provide a list of all payment facilities?"
#user = "Can i do foreign currency payments?"

user = "what is the eligibility of the Easy Money Card?" # Does not pick up Easy Money card
#user = "what is the eligibility of the Bendigo Easy Money Card?" # Does not pick up Easy Money card
#user = "Can the Bendigo Easy Money Card be used overseas?" # Does not pick up Easy Money card

#user = "Can changes be made to the terms and conditions?"
#user = "Does the document contain information about the Easy Money card"
#user = "are there any limits on my debit card?"
#user = "can i do progress payments on my building loan?"
#user = "who is liable for unauthorised transactions?"
#user = "can Foreign Currency Transactions be performed?"
#user = "How do you process pay anyone payments?"
#user = "What happens if accidently entered the incorrect amount for my pay anyone payments?"
#print(answer_question(user, df3))

# Existing code
ans = (answer_question(user, df3))
print(ans)




# -- get source --
# -- Extract from func (could make output of func)
#ans = (answer_question(user, df3))
question_embedding = model_trans.encode(user, convert_to_tensor=True).cpu()
similarities = df3['embeddings'].apply(lambda x: np.dot(question_embedding, x))
most_similar_indices = similarities.nlargest(10).index
most_similar_rows = df3.iloc[most_similar_indices]

# -- Get the target index
question_embedding1 = model_trans.encode(ans, convert_to_tensor=True).cpu()
similarities1 = most_similar_rows['embeddings'].apply(lambda x: np.dot(question_embedding1, x))
most_similar_indices1 = similarities1.nlargest(10).index

# -- Print source
pd.set_option('display.max_colwidth', None)
filtered_df = most_similar_rows.loc[most_similar_indices1]
#filtered_df = filtered_df[['Title', 'page', 'page_content']]
#filtered_df = filtered_df[['page_content', 'source']]
filtered_df = filtered_df[['page_content_org', 'Title']]
display(filtered_df)
